For the March 2024 Sky 1 and 2 datasets

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('Results/result_sky1.csv')

In [ ]:
# Ensure the "Date" column is in datetime format
data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%dT%H:%M:%S.%f")

# Function to correct a band's DataFrame
def correct_band_dataframe(df, band, t):
    # Find the reference date for this band
    object_count_per_date = df.groupby("Date")["Object"].nunique()
    reference_date = object_count_per_date.idxmax()
    
    # Initialize a DataFrame to hold the average magnitude differences
    avg_mag_diff = pd.DataFrame(columns=["Date", "Avg_Magnitude_Difference"])
    
    # Loop through each unique date in this band's DataFrame
    unique_dates = df["Date"].unique()

    for date in unique_dates:
        if date == reference_date:
            continue  # Skip the reference date itself

        airmass = df[df["Date"] == reference_date]["Airmass"].values[0]
        # Find common objects between the reference date and the current date
        ref_mag_col = f"Apparent magnitude {band} (instrumental)"
        common_stars = df[df["Date"] == reference_date][["Object", ref_mag_col]].merge(
            df[df["Date"] == date][["Object", ref_mag_col]], on="Object", suffixes=("_ref", "_other")
        )
        
        if common_stars.empty:
            # If no common stars, set average difference to zero
            avg_mag_diff = avg_mag_diff.append({
                "Date": date,
                "Airmass": airmass,
                "Avg_Magnitude_Difference": 0.0
            }, ignore_index=True)

        else:
            # Calculate the average magnitude difference
            mag_difference = common_stars[f"{ref_mag_col}_ref"] - common_stars[f"{ref_mag_col}_other"]
            avg_diff = mag_difference.mean()
            avg_mag_diff = avg_mag_diff.append({
                "Date": date,
                "Airmass": airmass,
                "Avg_Magnitude_Difference": avg_diff
            }, ignore_index=True)

    # Correct magnitudes for the current band
    new_mag_col = f"Corrected magnitude {band} (instrumental, {t})"

    for date in unique_dates:
        if date == reference_date:
            continue
        
        original_mag_col = f"Apparent magnitude {band} (instrumental)"
        row = avg_mag_diff[avg_mag_diff["Date"] == date]
        avg_diff = 0.0

        if not row.empty:
            avg_diff = row["Avg_Magnitude_Difference"].values[0]
        
        df.loc[df["Date"] == date, new_mag_col] = df.loc[df["Date"] == date, original_mag_col] + avg_diff

    # df[f"Apparent magnitude {band} (instrumental, {t})"] = df[f"Apparent magnitude {band} (instrumental)"]
    df[df["Date"] == reference_date][f"Corrected magnitude {band} (instrumental)"] = df[df["Date"] == reference_date][f"Apparent magnitude {band} (instrumental)"]
    df.drop(f"Apparent magnitude {band} (instrumental)", axis=1, inplace=True)
    # df[f'Airmass, {t}'] = df['Airmass']
    df.drop('Airmass', axis=1, inplace=True)

    return df

# Separate DataFrames by "Type"
types = data["Type"].unique()
type_dfs = {t: data[data["Type"] == t] for t in types}
# Correct each band's DataFrame for each type
corrected_band_dfs = {}

# Iterate by type first, then by band
for t in types:
    type_band_df = pd.DataFrame()
    
    if t == "ZWO":
        bands = ["R", "G", "B"]

    else:
        bands = ["R", "V", "B"]

    for band in bands:
        type_specific_band_df = type_dfs[t][["Object", "Date", "Airmass", f"Apparent magnitude {band} (instrumental)"]]
        # Drop rows with missing values in the apparent magnitude of that band
        type_specific_band_df = type_specific_band_df.dropna(subset=[f"Apparent magnitude {band} (instrumental)"])
        # Correct and concatenate results for this type and band
        type_specific_band_df = correct_band_dataframe(type_specific_band_df, band, t)
        type_band_df = pd.concat([type_band_df, type_specific_band_df])

    corrected_band_dfs[t] = type_band_df

joined_df = pd.DataFrame()

for t in types:
    if joined_df.empty:
        joined_df = corrected_band_dfs[t]

    else:
        joined_df = joined_df.merge(corrected_band_dfs[t], on=["Object", "Date"], how="outer")

In [ ]:
final_df = data.merge(joined_df, on=["Object", "Date"], how="outer")
final_df.to_csv('Results/result_sky1_corr.csv', index=False)